# Motive / Optitrack to OpenEphys alignment
Assumes a TTL from Motive to OpenEphys at the start of each recording. If it doesn't find that, will try to align by timestamps.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path

sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy/'])  # Add path to NeuroPy repository here
sys.path.extend(['/data/GitHub/NeuroPy/'])

from neuropy import core
from neuropy.io import (optitrackio,
                        dlcio,
                        )
from neuropy.io.neuroscopeio import NeuroscopeIO
from neuropy.io.binarysignalio import BinarysignalIO 
from neuropy.io.optitrackio import OptitrackIO
import neuropy.io.openephysio as oeio
from neuropy.core.epoch import Epoch


## Load in session data and Open Ephys timestamps

In [26]:
dir_use = Path("/data3/Psilocybin/Recording_Rats/Finn/2022_02_17_psilocybin/")
sess = core.ProcessData(dir_use)

# Create sync_df to ID start and end of each recording in concatenated .eeg file
sync_df = oeio.create_sync_df(dir_use)

# Get absolute time from start of first recording
sync_df['Tabs from start (s)'] = (sync_df['Datetime'] - sync_df['Datetime'].iloc[0]).dt.total_seconds()
sync_df

start time = 2022-02-17 10:33:51.810584-05:00
start time = 2022-02-17 10:45:04.949076-05:00
start time = 2022-02-17 11:44:40.100407-05:00


,Recording,Datetime,Condition,nframe_dat,dat_time,nframe_eeg,eeg_time,Tabs from start (s)
0,0,2022-02-17 10:33:51.810584-05:00,start,0,0.000000,0,0.0000,0.000000
1,0,2022-02-17 10:39:00.589217333-05:00,stop,9263359,308.778633,385973,308.7784,308.778633
2,1,2022-02-17 10:45:04.949076-05:00,start,9263360,308.778667,385974,308.7792,673.138492
3,1,2022-02-17 11:43:54.805042667-05:00,stop,115159039,3838.634633,4798293,3838.6344,4202.994459
4,2,2022-02-17 11:44:40.100407-05:00,start,115159040,3838.634667,4798294,3838.6352,4248.289823
5,2,2022-02-17 13:24:56.424640333-05:00,stop,295648767,9854.958900,12318698,9854.9584,10264.614056


## Next get OptiTrack recording start

### Method 1: Calculate from .tak file modification time and # frames in .avi file(no rigid body tracking or .csv file, ~millisecond precision?).

This method is used for synchronizing below...

In [66]:
from datetime import datetime
from neuropy.io.dlcio import DLC
import os
tak_file_dir = Path("/run/media/nkinsky/LWD_Backup/Nat/Psilocybin/Recording_Rats/Finn/2022_02_17_psilocybin/")
dlc_file_dir = Path("/data3/Psilocybin/Recording_Rats/Finn/2022_02_17_psilocybin/")
tak_files = sorted(tak_file_dir.glob("**/*.tak"))

mtimes = []
for tak_file in tak_files:
    mtimes.append(pd.Timestamp(datetime.fromtimestamp(os.path.getmtime(tak_file))))

dlc = DLC(dlc_file_dir)

opti_sr = dlc.SampleRate
mtimes

Using tracking file #0: /data3/Psilocybin/Recording_Rats/Finn/2022_02_17_psilocybin/1_baseline/Take 2022-02-17 10.33.54 AM-Camera 4 (#410110)DLC_resnet50_Psilocybin_DLCFeb13shuffle1_500000.h5
Using tracking file #1: /data3/Psilocybin/Recording_Rats/Finn/2022_02_17_psilocybin/2_psilocybin/Take 2022-02-17 10.45.11 AM-Camera 4 (#410110)DLC_resnet50_Psilocybin_DLCFeb13shuffle1_500000.h5
Multiple videos found - taking mean sample rate from all videos


[Timestamp('2022-02-17 13:24:52.349000')]

In [51]:
from neuropy.io.movie import tracking_movie
tz = "America/Detroit"
start_times = []
for tak_file, mtime in zip(tak_files, mtimes):
    try:
        movie_file = sorted(tak_file.parent.glob(f"{tak_file.stem}*.avi"))[0]
        mobj = tracking_movie(str(movie_file))
        start_times.append((mtime - pd.Timedelta(mobj.nframes / dlc.SampleRate, unit="sec")).tz_localize(tz))
    except IndexError:
        start_times.append(np.nan)

start_times

[Timestamp('2022-02-17 10:45:12.649000-0500', tz='America/Detroit')]

In [52]:
start_time_df = pd.DataFrame({"file": [tak_file.stem for tak_file in tak_files], 
                              "mod_time": mtimes,
                              "inferred_start_time": start_times,
                              "Recording": np.arange(len(tak_files))})

start_time_df.to_csv(sess.basepath / "inferred_start_time.csv")
start_time_df

,file,mod_time,inferred_start_time,Recording
0,Take 2022-02-17 10.45.11 AM,2022-02-17 13:24:52.349,2022-02-17 10:45:12.649000-05:00,0


In [98]:
# Manually insert # frames and sample rate for file above
start_time_df["SampleRate"] = dlc.SampleRate
start_time_df["nframes"] = dlc.nframes[1] # 

start_time_df.to_csv(sess.basepath / "inferred_start_time.csv")
start_time_df

,file,mod_time,inferred_start_time,Recording,SampleRate,nframes
0,Take 2022-02-17 10.45.11 AM,2022-02-17 13:24:52.349,2022-02-17 10:45:12.649000-05:00,0,30.0,287391


### Method 2: From .csv file output from .tak file (microsecond precision)
Note that lag from start time logged in .tak file to actual recording start time can be ~1.5 seconds.  

Not used below.

In [24]:
csv_path = "/data2/Alternation/Recording_Rats/Finn/2022_02_01_alternation2/1_alternation/"
opti = OptitrackIO(csv_path)
tstart_from_csv = opti.datetime_array[0]
tstart_from_csv

/data2/Alternation/Recording_Rats/Finn/2022_02_01_alternation2/1_alternation/Take 2022-02-01 11.42.46 AM.csv


Timestamp('2022-02-01 11:42:47.565000')

## Method 1: Motive start triggers some TTLs (but not all) at start only.
Example: Motive start triggers miniscope data acquisition which delivers a TTL for each frame acquired. Therefore the start of a TTL block can coincide with a Motive recording start, but in case of a disconnect we may a miniscope recording restart without restarting Motive.

In [63]:
# Load in ALL TTLs
ttl_start_chan_use = 1
ttl_df = oeio.load_all_ttl_events(dir_use)
ttl_df = ttl_df[ttl_df.channel_states.abs() == ttl_start_chan_use]
ttl_df.head(5)

,channel_states,timestamps,datetimes,sample_number,event_name,Recording
0,-1,82316,2022-02-17 10:33:54.554450667-05:00,82316,,0
1,1,84406,2022-02-17 10:33:54.624117333-05:00,84406,,0
2,-1,86359,2022-02-17 10:33:54.689217333-05:00,86359,,0
3,1,88384,2022-02-17 10:33:54.756717333-05:00,88384,,0
4,-1,90431,2022-02-17 10:33:54.824950667-05:00,90431,,0


In [117]:
row["inferred_start_time"]

Timestamp('2022-02-17 10:45:12.649000-0500', tz='America/Detroit')

In [119]:
pd.Series(row["inferred_start_time"] +  pd.to_timedelta(np.arange(row["nframes"])/row["SampleRate"], unit='s'))

Timestamp('2022-02-17 10:45:12.649000-0500', tz='America/Detroit')

In [122]:
# Workhorse code to align each motive recording to each corresponding block of TTLs in OE

buffer_sec = 1 # time buffer in seconds for start / stop detection
oe_sr = 30000  # OpenEphys sample rate
buffer_td = pd.Timedelta(buffer_sec, unit="sec")

opti_times_df = []
for idr, row in start_time_df.iterrows():
    
    # First grab timestamp for Optitrack/Motive recording start
    opti_start = row["inferred_start_time"]
    
    # Next ID ephys recording start based on closest TTL to start
    rec_bool = ((sync_df[sync_df.Condition == "start"]["Datetime"] - buffer_td)  < opti_start).values & ((sync_df[sync_df.Condition == "stop"]["Datetime"] + buffer_td) > opti_start).values
    oe_rec_num_use = sync_df[sync_df.Condition == "start"][rec_bool]["Recording"].iloc[0]
    oe_rec_num_use
    
    # Grab ttls from specific recording
    ttl_rec = ttl_df[ttl_df.Recording == oe_rec_num_use]
    
    oe_start_inds = np.where((ttl_rec["timestamps"].diff() > (buffer_sec * oe_sr)) |  np.isnan(ttl_rec["timestamps"].diff()))[0]
    start_ind = oe_start_inds[np.where((ttl_rec.iloc[oe_start_inds]["datetimes"] - opti_start).dt.total_seconds().abs() < buffer_sec)[0]]
    assert len(start_ind) == 1, "multiple candidate start times found in OE TTLs, adjust buffers and try again"
    start_ttl = ttl_rec.iloc[start_ind[0]]
    
    # Print stuff to screen as sanity check
    # print(f"MS start = {ms_start_ts}")
    # print(f"Corresponding TTL time in OE = {start_ttl['datetimes']}")

    # Calculate delta between miniscope time start and ttl received in OE
    start_delta = start_ttl['datetimes'] - opti_start
    # print(f"start_delta_sec={start_delta.total_seconds()}")

    # Now adjust each timestamp in miniscope timestamps by start offset from OE TTL 
    prior_oe_rec_start_dt = sync_df[(sync_df.Recording == oe_rec_num_use) & (sync_df.Condition == "start")].iloc[0]["Datetime"]
    prior_oe_rec_start_comb_time = sync_df[(sync_df.Recording == oe_rec_num_use) & (sync_df.Condition == "start")].iloc[0]["eeg_time"]
    opti_ts = pd.Series(row["inferred_start_time"] +  pd.to_timedelta(np.arange(row["nframes"])/row["SampleRate"], unit='s'))
    opti_times_eeg_align = pd.DataFrame((opti_ts + start_delta - prior_oe_rec_start_dt).dt.total_seconds() + prior_oe_rec_start_comb_time)
    
    # Save recording number
    opti_times_eeg_align["Recording"] = row["Recording"]
    
    opti_times_df.append(opti_times_eeg_align)

# Get output times of all miniscope timestamps now referenced to the eeg time.
opti_ts_align_df = pd.concat(opti_times_df, axis=0)
opti_ts_align_df

,0,Recording
0,316.502633,0
1,316.535967,0
2,316.569300,0
3,316.602633,0
4,316.635967,0
...,...,...
287386,9896.035967,0
287387,9896.069300,0
287388,9896.102633,0
287389,9896.135967,0


### NRK start here!

In [ ]:
# Create a dataframe for the start and stop time of each recording in EEG time. 
ms_starts = ms_ts_align_df.loc[0]["Timestamps"].values
ms_stops = np.hstack((ms_ts_align_df[(ms_ts_align_df["Timestamps"].diff(-1) < - buffer_sec)]["Timestamps"].values, 
                       ms_ts_align_df.iloc[-1]["Timestamps"]))

ms_rec_epochs = Epoch(pd.DataFrame({"start": ms_starts, "stop": ms_stops, "label": "ms_rec_after_align"}))
ms_rec_epochs

In [ ]:
# Construct time epochs of miniscope recordings from TTL series. Assumes delta between timestamps from TTLs > ttl_rec_thresh
ttl_rec_thresh = 1  # Any TTLs greater than this # seconds apart will be considered from a separate recording
rec_label = "miniscope_from_ttl"
sr_ttl = 30000

# Get absolute time from first recording start for each TTL - will chop out other isolated TTLs (noise) that occur
## Rule for start - diff = nan or diff > thresh
## Rule for stop: diff(-1) < -thresh
dt_ttl_sec = (ttl_df["datetimes"] - sync_df.iloc[0]["Datetime"]).dt.total_seconds()
rec_start_inds = np.where((dt_ttl_sec.diff() > ttl_rec_thresh) | np.isnan(dt_ttl_sec.diff()))[0]
rec_stop_inds = np.where((dt_ttl_sec.diff(-1) < -ttl_rec_thresh) | np.isnan(dt_ttl_sec.diff(-1)))[0]

# Interpolate start and stop back to combined recording time
rec_start_t_comb = np.interp(dt_ttl_sec.iloc[rec_start_inds].values, sync_df["Tabs from start (s)"], sync_df["eeg_time"])
rec_stop_t_abs = np.interp(dt_ttl_sec.iloc[rec_stop_inds].values, sync_df["Tabs from start (s)"], sync_df["eeg_time"])
# rec_stop_inds = np.hstack((rec_stop_inds, ttl_df.shape[0] - 1))

rec_epochs_from_ttl = Epoch(pd.DataFrame({"start": rec_start_t_comb, "stop": rec_stop_t_abs, "label": rec_label}))

rec_ttl_align_df = []
zero_length_bool = (rec_stop_inds - rec_start_inds) == 0  # Chop out any isolated TTLs
for idr, (start_ind, stop_ind) in enumerate(zip(rec_start_inds[~zero_length_bool], 
                                                rec_stop_inds[~zero_length_bool])):
    rec_ttl_times_use = np.interp(dt_ttl_sec.iloc[start_ind:stop_ind].values, sync_df["Tabs from start (s)"], sync_df["eeg_time"])
    rec_ttl_align_df.append(pd.DataFrame({"ttl_sec_from_eeg_start": rec_ttl_times_use, "Recording": idr}))
rec_ttl_align_df = pd.concat(rec_ttl_align_df, axis=0)

# print(rec_start_inds)
# print(rec_stop_inds)
rec_epochs_from_ttl

## Sanity checks


Plot epochs created from both miniscope times and OE TTLs times to make sure they match.

In [ ]:
import neuropy.plotting.epochs as plt_epochs
_, ax = plt.subplots(2, 1, sharex=True)
plt_epochs.plot_epochs(rec_epochs_from_ttl[rec_epochs_from_ttl.durations > 0], ax=ax[0])
plt_epochs.plot_epochs(ms_rec_epochs, ax=ax[1])

Print out # frames you expect and check to see if any don't match

In [ ]:
# Total # frames in miniscope data
np.load(dir_use / "Miniscope_combined/minian/C.npy").shape

In [ ]:
# total number frames including corrupt frames
np.load(dir_use / "Miniscope_combined/minian/good_frames_bool.npy").shape

## Investigate dropped frames here - will vary for each session

Example here illustrates general procedure to identify shifts in timestamps offsets.

In [ ]:
# First identify any sessions with mismatched frame #s
nframes_ttl, nframes_ms = [], []
for nrec in rec_ttl_align_df.Recording.unique():
    nframes_ttl.append((rec_ttl_align_df.Recording == nrec).sum())
    nframes_ms.append((ms_ts_align_df[~ms_ts_align_df.Pre_rec_buffer].Recording == nrec).sum())
nframes_ttl = np.array(nframes_ttl)
nframes_ms = np.array(nframes_ms)

mismatches = np.where((nframes_ttl - nframes_ms) != 0)[0]
print(f"Mismatched #frames in MS recordings # {mismatches}")

In [ ]:
mismatch_rec = 1
ttl_mismatch = rec_ttl_align_df[rec_ttl_align_df.Recording == mismatch_rec]
ms_ts_mismatch = ms_ts_align_df[(ms_ts_align_df.Recording == mismatch_rec) & ~ms_ts_align_df.Pre_rec_buffer]
nframes_min = np.min((ttl_mismatch.shape[0], ms_ts_mismatch.shape[0]))
diffs = ttl_mismatch["ttl_sec_from_eeg_start"].iloc[0:nframes_min].values - ms_ts_mismatch['Timestamps'].iloc[0:nframes_min].values
diffs

In [ ]:
# Get miniscope sample rate
from neuropy.io.miniscopeio import get_recording_metadata
import re

rec_metadata, vid_metadata, _ = get_recording_metadata("/data2/Trace_FC/Recording_Rats/Finn/2022_01_20_training/2_training/Finn/gobears/2022_01_20/12_49_22")
sr_spec = 1 / float(re.sub("FPS", "", vid_metadata['frameRate']))
print(f"SR specified = {sr_spec}")

# Calculate effective sample rate, make sure it matches above
ms_rec = ms_ts_align_df[(ms_ts_align_df.Recording == mismatch_rec) & ~ms_ts_align_df.Pre_rec_buffer]
sr_actual = ms_rec.Timestamps.diff().mean()
print(f"SR actual = {sr_actual}")

In [ ]:
# Plot differences between timestamps and overlay miniscope sampling rate intervals to identify missing frames!
# Every time it jumps a box below you have a missing frame
%matplotlib widget
_, ax = plt.subplots()
ax.plot(diffs)
for sr_interval in np.arange(np.floor(np.min(diffs) / sr_spec), np.ceil(np.max(diffs) / sr_spec) + 1, 1):
    ax.axhline(sr_interval*sr_spec, color='r', linestyle='--')

drop_bool = np.abs(np.diff(diffs)) > sr_spec
drop_inds = np.where(drop_bool)[0]
ax.plot(drop_inds, diffs[:-1][drop_inds + 1], 'g*')
ax.set_ylabel("Lag between TTL and MS timestamp (s)")
ax.set_xlabel("Combined Miniscope Frame # (w/o pre-record buffered frames)")
ax.set_title(f"Lags from mismatch recording #{mismatch_rec} BEFORE fixing")

print(f"Approximate dropped frame locations at {drop_inds}")

# NRK pick up here
Will need to insert a boolean into the TTL df for dropped frames at the appropriate recording, then clip those out and plot the differences between timestamps to check (both in the main recording AND across the whole recording).
- Also should output approximate slop in aligning the data, which is the mean difference between timestamps after alignment - not clear if that is due to hardware lag or actual capture times.

In [ ]:
# Now play around below to identify indices to drop - should have everything around the same time difference
dropped_inds = [2456, 2457, 2458, 2661, 2662]
ttl_clipped = ttl_mismatch.drop(index=dropped_inds)
diffs_fixed = ttl_clipped["ttl_sec_from_eeg_start"].values - ms_ts_mismatch["Timestamps"].values
_, ax = plt.subplots()
ax.plot(diffs_fixed)
ax.set_ylabel("Lag between TTL and MS timestamp (s)")
ax.set_xlabel("Combined Miniscope Frame # (w/o pre-record buffered frames)")
ax.set_title(f"Lags from mismatch recording #{mismatch_rec} after fixing")

In [ ]:
# Add in dropped frame info to aligned ttl DataFrame
dropped_bool = np.isin(rec_ttl_align_df.index, dropped_inds) & (rec_ttl_align_df.Recording == mismatch_rec)
rec_ttl_align_df["Dropped_by_ms"] = dropped_bool
rec_ttl_align_df

In [ ]:
# Check full alignment now! Should show mostly very small differences between frame times with only periodic blips where one frame
# was captured with some jitter.
full_diffs = rec_ttl_align_df[~rec_ttl_align_df.Dropped_by_ms]["ttl_sec_from_eeg_start"].values - ms_ts_align_df[~ms_ts_align_df.Pre_rec_buffer]['Timestamps'].values

_, ax = plt.subplots()
ax.plot(full_diffs)
ax.set_ylabel("Lag between TTL and MS timestamp (s)")
ax.set_xlabel("Combined Miniscope Frame # (w/o pre-record buffered frames)")

## Save aligned timing dataframes! Dump TTL timing into ms dataframe.
Can either use ms data directly OR TTL timing later during analysis.

In [ ]:
# Insert TTL times into aligned miniscope timestamp dataframe
ms_ts_align_df.loc[~ms_ts_align_df.Pre_rec_buffer, "TTL_time"] = rec_ttl_align_df[~rec_ttl_align_df.Dropped_by_ms]["ttl_sec_from_eeg_start"].values

# Save to .npy file and/or csv file
np.save(sess.filePrefix.with_suffix(".ms_times_aligned.npy"), ms_ts_align_df, allow_pickle=True)
ms_ts_align_df.to_csv(sess.filePrefix.with_suffix(".ms_times_aligned.csv"))
ms_ts_align_df